In [1]:
import torch
import os

from datasets import MuscimaObjects

ModuleNotFoundError: No module named 'datasets'

In [5]:
torch.cuda.get_device_name()

'Tesla T4'

In [4]:
!

/scratch/users/cakarsubasi18/JupyterWorkspace


In [ ]:
# load the measures model
SAVED_MODEL_MEASURES = os.path.join('saved_models', 'measures_5_layers.pt')
model = torch.load(SAVED_MODEL_MEASURES)
# load the object annotations
dataset = MuscimaObjects.MuscimaObjects(muscimaroot, label_list_pitch_only, )
dataset_test = MuscimaObjects.MuscimaObjects(muscimaroot, label_list_pitch_only, transforms=get_transform(train=False))
#splits
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-40])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-40:])

# get the 